In [1]:
import json
import os
from ast import literal_eval

import pandas as pd
from dotenv import load_dotenv
from langchain.output_parsers import OutputFixingParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm

In [ ]:
load_dotenv("../.env")

True

In [ ]:
df = pd.read_csv("../data/train_v2.0.1.csv")

df["choices"] = df["choices"].apply(literal_eval)
df["question_plus"] = df["question_plus"].fillna("")
df.head()

,id,paragraph,question,choices,answer,question_plus
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"[이곳에 대장도감을 설치하여 재조대장경을 만들었다., 지눌이 이곳에서 수선사 결사운...",4,
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"[살수에서 수의 군대를 물리쳤다 ., 김춘추 의 신라 왕위 계승을 지원하였다 ., ...",2,
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"[사창제를 실시하였다 ., 대전회통을 편찬하였다 ., 비변사의 기능을 강화하였다 ....",3,
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"[나 당연합군의 공격으로 멸망하였다 ., 9주 5소경의 지방 행정 구역을 두었다 ....",3,


In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
def build_input(paragraph, question, choices, question_plus="", answer=""):
    question_plus_string = f"\n\n<보기>:\n{question_plus}" if question_plus else ""
    question = f"{question}{question_plus_string}"
    choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(choices)])
    answer = f"{answer} - {choices[answer - 1]}"
    return {"paragraph": paragraph, "question": question, "choices": choices_string, "answer": answer}

In [ ]:
prompt = """### 지시사항
아래 주어진 정보를 바탕으로 질문에 대한 정답을 논리적으로 도출하는 풀이를 만드세요. 
결과는 JSON 형식으로 반환하며, reasoning에 한 문장으로 작성되어야 합니다. 
reasoning은 다음의 구조를 따라야 합니다:
1. 문단(Paragraph)에서 핵심 정보를 파악합니다.
2. 질문(Question)과 보기(Choices)를 비교하여 정답을 논리적으로 도출합니다.
3. 최종 결론을 명확히 한 문장으로 요약합니다.

#### Paragraph
{paragraph}

#### Question
{question}

#### Choices
{choices}

#### Answer
{answer}

### 요청 사항
1. 문단 분석: Paragraph에서 질문 해결에 필요한 핵심 정보를 파악합니다.
2. 질문 분석: Question과 Choices를 세부적으로 비교하며 정답을 도출합니다.
3. 논리적 도출: reasoning을 "문단과 질문의 정보, 선택지의 비교, 최종 결론" 순으로 논리적으로 작성하세요.
4. JSON 형식 반환: 최종 결과를 JSON 형식으로 반환하며, reasoning은 한 문장으로 작성하세요.

### 제약 조건
reasoning은 정답 도출 과정을 포함하되, 간결하고 논리적으로 서술해야 합니다.
선택지와 문단 간 매칭 과정이 reasoning에 드러나야 합니다.
"""

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        HumanMessage(
            content=prompt.format(
                **build_input(
                    paragraph="국민의 권리와 의무에 관한 사항을 법률로 정해야 한다는 A에 따르면  법적 안정성과 예측 가능성을 확보하기 위해 법률 그 자체에 의한 통치가  강조되어,  정당하지 않은 법률에 의한 통치도 가능하였다 .  반면 B는 법률의  내용도 정의에 부합할 것을 요구하여,  입법자도 기본권 보장이라는 헌법의  이념에 구속되도록 한다.",
                    question="법치주의의 유형 A,  B에 대한 설명으로 옳은 것은?",
                    choices=[
                        "A는 행정 작용이 의회가 제정한 |법률에 근거해야 한다고 본다.",
                        "B는 국민의 자유와 권리 보장보다 통치의 합법성을 중시한다.",
                        "B와  달리 A 는 위헌 법률 심사제의 도입을 옹호한다.",
                        "A와 B  모두 명목상 법률에 의해 통치 질서가 확보되는 것만으로도  기본권 보장이 이루어진다고 본다.",
                    ],
                    answer=1,
                )
            )
        ),
        AIMessage(
            content=json.dumps(
                {
                    "reasoning": 'A는 법률 그 자체에 의한 통치를 강조하며, 법적 안정성과 예측 가능성을 중시하는 반면, B는 법률의 내용이 정의에 부합하고 기본권 보장을 중시한다는 점에서 실질적 정의를 강조합니다. 선택지 중 "A는 행정 작용이 의회가 제정한 법률에 근거해야 한다고 본다"는 A의 형식적 법치주의 특징을 정확히 반영합니다. 반면 다른 선택지는 B의 실질적 정의 강조나 문단 내용과 맞지 않습니다. 그렇기 때문에 정답은 1번입니다.',
                },
                ensure_ascii=False,
            )
        ),
        HumanMessagePromptTemplate.from_template(prompt),
    ]
)

In [ ]:
output_parser = JsonOutputParser()

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini", temperature=0, max_tokens=None, timeout=None, max_retries=2, api_key=OPENAI_API_KEY
)
fixing_parser = OutputFixingParser.from_llm(parser=output_parser, llm=llm)

chain = chat_prompt_template | llm | fixing_parser

In [ ]:
row = df.iloc[0]
print(
    chat_prompt_template.invoke(
        build_input(
            paragraph=row["paragraph"],
            question=row["question"],
            choices=row["choices"],
            question_plus=row["question_plus"],
            answer=row["answer"],
        )
    )
    .messages[2]
    .content
)

### 지시사항
아래 주어진 정보를 바탕으로 질문에 대한 정답을 논리적으로 도출하는 풀이를 만드세요. 
결과는 JSON 형식으로 반환하며, reasoning에 한 문장으로 작성되어야 합니다. 
reasoning은 다음의 구조를 따라야 합니다:
1. 문단(Paragraph)에서 핵심 정보를 파악합니다.
2. 질문(Question)과 보기(Choices)를 비교하여 정답을 논리적으로 도출합니다.
3. 최종 결론을 명확히 한 문장으로 요약합니다.

#### Paragraph
상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服) 절차에 대하여 논한 것이 신과는 큰 차이가 있었습니다 . 장자를 위하여 3년을 입는 까닭은 위로 ‘정체(正體)’가 되기 때문이고 또 전 중(傳重: 조상의 제사나 가문의 법통을 전함)하기 때문입니다 . …(중략) … 무엇보다 중요한 것은 할아버지와 아버지의 뒤를 이은 ‘정체’이지, 꼭 첫째이기 때문에 참 최 3년 복을 입는 것은 아닙니다 .”라고 하였다 .－현종실록 －

#### Question
상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?

<보기>:
ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 .ㄷ.정조 시기에 탕평 정치의 한 축을 이루었다 .ㄹ.이 이와 성혼의 문인을 중심으로 형성되었다.

#### Choices
1 - ㄱ, ㄴ
2 - ㄱ, ㄷ
3 - ㄴ, ㄹ
4 - ㄷ, ㄹ

#### Answer
2 - ㄱ, ㄷ

### 요청 사항
1. 문단 분석: Paragraph에서 질문 해결에 필요한 핵심 정보를 파악합니다.
2. 질문 분석: Question과 Choices를 세부적으로 비교하며 정답을 도출합니다.
3. 논리적 도출: reasoning을 "문단과 질문의 정보, 선택지의 비교, 최종 결론" 순으로 논리적으로 작성하세요.
4. JSON 형식 반환: 최종 결과를 JSON 형식으로 반환하며, reasoning은 한 문장으로 작성하세요.

### 제약 

In [ ]:
row = df.iloc[0]
chain.invoke(
    build_input(
        paragraph=row["paragraph"],
        question=row["question"],
        choices=row["choices"],
        question_plus=row["question_plus"],
        answer=row["answer"],
    )
)

{'reasoning': '문단에서는 상소한 인물이 상복 절차에 대한 자신의 견해를 밝히고 있으며, 이는 특정 붕당의 전통이나 가치관과 관련이 있을 수 있습니다. 질문에서 요구하는 붕당의 설명은 역사적 사실에 기반해야 하며, 선택지 ㄱ은 기 사환국의 정권 장악을, ㄷ은 정조 시기의 탕평 정치와 관련이 있습니다. 따라서 ㄱ과 ㄷ은 역사적 사실로서 옳은 설명입니다. 반면 ㄴ과 ㄹ은 상소한 인물의 붕당과 관련된 정보가 부족하므로 정답이 아닙니다. 따라서 정답은 2번입니다.'}

In [ ]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    result = ""
    try:
        result = chain.invoke(
            build_input(
                paragraph=row["paragraph"],
                question=row["question"],
                choices=row["choices"],
                question_plus=row["question_plus"],
                answer=row["answer"],
            )
        )
        df.loc[i, "reasoning"] = result["reasoning"]
    except Exception:
        print(result)
        print(row["id"])
        print("=========================================")

100%|██████████| 400/400 [17:30<00:00,  2.63s/it] 


In [ ]:
df.to_csv("../data/train_v3.0.0.csv", index=False)

In [ ]:
df.head()

,id,paragraph,question,choices,answer,question_plus,reasoning
0,generation-for-nlp-2089,프랑수아 올랑드 프랑스 대통령은 132명의 목숨을 앗아간 ‘11·13 파리 연쇄테러...,프랑수아 올랑드 대통령이 전쟁을 선포한 테러 단체의 이름은 무엇인가?,"[알카에다, 이슬람국가(IS), 탈레반, 하마스, 보코하람]",2,,문단에서 프랑수아 올랑드 대통령이 전쟁을 선포한 테러 단체는 '이슬람국가(IS)'로...
1,generation-for-nlp-2001,강원 원주시가 봉화산2지구에서 공동주택용지를 경쟁입찰 방식으로 공급하려다 돌연 수의...,원주시가 봉화산2지구에서 공동주택용지를 공급하기 위해 처음에 어떤 방식으로 계약을 ...,"[경쟁입찰 방식, 수의계약 방식, 직접 계약 방식, 공모 방식, 임의 계약 방식]",1,,문단에서 원주시는 봉화산2지구 공동주택용지 공급을 위해 처음에 경쟁입찰 방식으로 계...
2,generation-for-nlp-2002,한동안 공급이 뜸했던 경기 파주·포천·양주·의정부 등 수도권 북부지역에서 아파트 분...,수도권 북부지역에서 아파트 분양이 증가하고 있는 주된 이유는 무엇인가?,"[전세난과 저금리로 인한 실수요 증가, 부동산 가격 상승으로 인한 투자 수요 증가,...",1,,문단에서는 수도권 북부지역에서 아파트 분양이 증가하는 이유로 전세난과 저금리로 인한...
3,generation-for-nlp-2003,지난 3월 은행 예금회전율이 최근 3년 사이 가장 높은 수준으로 상승했다. 전문가들...,3월 은행 예금회전율이 최근 3년 사이 가장 높은 수준으로 상승한 이유는 무엇인가?,"[은행에 머물던 돈이 주식과 부동산으로 이동하고 있기 때문, 소비자들이 예금을 늘리...",1,,문단에서는 3월 은행 예금회전율이 상승한 이유로 은행에 머물던 자금이 주식과 부동산...
4,generation-for-nlp-2835,지난해 정부가 발의한 ‘분양가상한제 탄력 적용(사실상 폐지) 법안’이 이달에도 국회...,분양가 상한제가 처음 도입된 정부는 어느 정부인가?,"[노무현 정부, 이명박 정부, 박근혜 정부, 문재인 정부, 윤석열 정부]",1,,문단에서 분양가 상한제가 노무현 정부 시절에 도입되었다고 명시되어 있습니다. 질문은...


In [ ]:
print(df.iloc[0]["reasoning"])

문단에서 프랑수아 올랑드 대통령이 전쟁을 선포한 테러 단체는 '이슬람국가(IS)'로 명시되어 있습니다. 질문은 이 테러 단체의 이름을 묻고 있으며, 선택지 중에서 '이슬람국가(IS)'가 정확히 일치합니다. 따라서 정답은 2번입니다.


End.
